# Python 实验3（进阶）

## 使用海龟绘制给定照片或艺术作品

**总共用到：** OpenCV + PyPotrace + turtle

1. OpenCV: 处理图像，获取轮廓等信息以确定如何画图。

2. [PyPotrace](https://pypi.org/project/pypotrace/#description): 将给定 png/jpg/bmp 等位图源文件矢量化。

3. turtle: 最终调用该库将图片画出来。

In [31]:
import turtle
import cv2
from matplotlib import pyplot as plt

In [32]:
def imshow(img):
    plt.imshow(img)
    plt.show()

def cvshow(img):
    cv2.imshow('img', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [44]:
img = cv2.imread('0.jpg')
img = cv2.pyrDown(img)
cvshow(img)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is n

In [50]:
img_ctr = cv2.Canny(img, 80, 150)
cvshow(img_ctr)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is n

In [47]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
cvshow(thresh)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is n

In [48]:
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

In [49]:
# Pass the image to draw, contours, contour indices, color mode, thickness of stroke
# Note that copying is necessary, or the original image will change...
draw_img = img.copy()
res = cv2.drawContours(draw_img, contours, -1, (0, 0, 255), 2)
cvshow(res)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is n

In [39]:
thresh_ctr = cv2.Canny(thresh, 80, 150)
cvshow(thresh_ctr)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is not the object's thread (0x2454e430).
Cannot move to target thread (0x2327c340)

QObject::moveToThread: Current thread (0x2327c340) is n